<a href="https://colab.research.google.com/github/brinda-lax/Elite-Tech/blob/main/Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install dask[complete]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.6 MB/s eta 0:00:00


## Reading the Dataset##


Here Im taking **Newyork** taxi Data set from the below mentioned url in



paraquet Format

In [2]:
import dask.dataframe as dd

url = "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet"

df = dd.read_parquet(url, engine="pyarrow")

In [3]:
df.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

Checking For Data size and columns

In [5]:
num_rows = df.map_partitions(len).compute().sum()


In [6]:
num_rows

np.int64(3066766)

In [7]:
df.info()

<class 'dask_expr.DataFrame'>
Columns: 19 entries, VendorID to airport_fee
dtypes: datetime64[us](2), float64(12), int64(4), string(1)

## Analysis

 ## Ride Trends & Revenue Overview

 Q1)What is the monthly trend of total rides over the past year?

In [8]:
df['tpep_pickup_datetime'] = dd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime']=dd.to_datetime(df['tpep_dropoff_datetime'])

In [9]:
max_date = df['tpep_pickup_datetime'].max().compute()
max_date

Timestamp('2023-02-01 00:56:53')

In [10]:
min_date = df['tpep_pickup_datetime'].min().compute()
min_date

Timestamp('2008-12-31 23:01:42')

In [11]:
from datetime import datetime,timedelta


In [12]:
start_date = max_date - timedelta(days=365)

In [13]:
start_date

Timestamp('2022-02-01 00:56:53')

In [14]:
q1_filtered_df = df[(df['tpep_pickup_datetime'] >= start_date) &
                 (df['tpep_pickup_datetime'] < max_date)]

In [15]:
q1_filtered_df['tpep_pickup_datetime'].max().compute()

Timestamp('2023-02-01 00:13:10')

In [16]:
q1_filtered_df['pickup_month']=q1_filtered_df['tpep_pickup_datetime'].dt.to_period('M').astype(str)

In [17]:
q1_filtered_df.shape[0].compute()


3066763

In [18]:
monthly_rides=q1_filtered_df.groupby('pickup_month').size().compute().reset_index(name='Count')

In [19]:
monthly_rides

,pickup_month,Count
0,2022-10,11
1,2022-12,25
2,2023-01,3066718
3,2023-02,9


That means almost 100% of the rides are from January 2023,
and only a few stray rides exist outside that month (like 11 in Oct 2022, 25 in Dec 2022, etc.).

Q2) What is the average fare amount over time in jan 2023?


In [20]:
jan_df = df[(df['tpep_pickup_datetime'] >= '2023-01-01') & (df['tpep_pickup_datetime'] < '2023-02-01')]


In [21]:
jan_df['pickup_date']=jan_df['tpep_pickup_datetime'].dt.date

In [23]:
avg_fare=jan_df.groupby('pickup_date')['fare_amount'].mean().compute().reset_index(name='Average_fare/day')
avg_fare.sort_values(by='Average_fare/day',ascending=False)

,pickup_date,Average_fare/day
1,2023-01-02,21.960321
0,2023-01-01,21.869196
2,2023-01-03,19.999048
15,2023-01-16,19.824897
7,2023-01-08,19.755304
3,2023-01-04,18.977018
28,2023-01-29,18.976934
21,2023-01-22,18.910542
8,2023-01-09,18.830372
16,2023-01-17,18.765341


Q3)What is the total revenue generated from each pickup location (PULocationID)?




In [24]:
total_amount_location=jan_df.groupby('PULocationID')['total_amount'].sum().compute().reset_index(name='Total Amount')

In [25]:
total_amount_location=total_amount_location.sort_values(by='Total Amount',ascending=False)

In [26]:
total_amount_location.head(5)

,PULocationID,Total Amount
124,132,11926574.76
130,138,5628923.24
153,161,3155541.57
228,237,2888515.19
227,236,2830541.55


Q3)What is the cancellation rate per location? (If cancellation can be inferred from trip_distance == 0 or missing dropoff)


In [27]:
jan_df_cancelled=jan_df[(jan_df['trip_distance']==0)| (jan_df['tpep_dropoff_datetime'].isna())]

In [28]:
jan_df_rides=jan_df['PULocationID'].value_counts().compute().reset_index(name='total_rate')

In [29]:
jan_df_cancelled_rides=jan_df_cancelled['PULocationID'].value_counts().compute().reset_index(name='cancelled_rate')

In [30]:
total_rides=jan_df_rides.merge(jan_df_cancelled_rides,on='PULocationID',how='left')

In [31]:
total_rides['cancelled_rate']=total_rides['cancelled_rate'].fillna(0)

In [32]:
total_rides['cancellation_rate']=(total_rides['cancelled_rate']/total_rides['total_rate'])

In [33]:
total_rides=total_rides.sort_values(by='cancellation_rate',ascending=False)

In [34]:
total_rides.head(10)

,PULocationID,total_rate,cancelled_rate,cancellation_rate
178,187,2,2.0,1.000000
107,115,4,4.0,1.000000
184,193,2713,2262.0,0.833763
28,30,12,10.0,0.833333
198,207,219,178.0,0.812785
0,1,408,324.0,0.794118
242,251,7,5.0,0.714286
256,265,1646,1122.0,0.681652
102,109,3,2.0,0.666667
137,145,2434,1429.0,0.587099


 ## Driver Performance
Purpose: Assess how different drivers/vendors perform.

Q1)What is the **average trip** duration by VendorID?






Here im going for **jan 2023**

In [35]:
jan_df['tpep_pickup_datetime']=dd.to_datetime(jan_df['tpep_pickup_datetime'])
jan_df['tpep_dropoff_datetime']=dd.to_datetime(jan_df['tpep_dropoff_datetime'])


In [36]:
jan_df['duration_in_minutes']=(jan_df['tpep_dropoff_datetime']-jan_df['tpep_pickup_datetime']).dt.total_seconds()/60

In [37]:
avg_trip_duration=jan_df.groupby('VendorID')['duration_in_minutes'].mean().compute().reset_index(name='Avg Duration')

In [38]:
avg_trip_duration=avg_trip_duration.sort_values(by='Avg Duration',ascending=False)

In [39]:
round(avg_trip_duration,2)

,VendorID,Avg Duration
1,2,16.07
0,1,14.57


Q2)How many trips are handled per VendorID?


In [40]:
count_of_trips=jan_df['VendorID'].value_counts().compute().reset_index(name='No of rides')
count_of_trips=count_of_trips.sort_values(by='No of rides',ascending=False)

In [ ]:
count_of_trips

Q3) Which VendorIDs have the highest delays (if expected time is estimable)?


Assuming average speed as 20kmph

In [41]:
avg_speed_km_per_min = 20 / 60
jan_df['expected_duration_min'] =(jan_df['trip_distance'] / avg_speed_km_per_min)

In [42]:
jan_df['expected_duration_min'].head()

,expected_duration_min
0,2.91
1,3.30
2,7.53
3,5.70
4,4.29


In [43]:
jan_df['duration_in_minutes'].head()

,duration_in_minutes
0,8.433333
1,6.316667
2,12.750000
3,9.616667
4,10.833333


In [44]:
jan_df['delay'] = jan_df['duration_in_minutes'] - jan_df['expected_duration_min']


In [45]:
jan_df_delay_VendorID=jan_df.groupby('VendorID')['delay'].mean().compute().reset_index(name='Avg delay per VendorID')

In [46]:
jan_df_delay_VendorID.sort_values(by='Avg delay per VendorID',ascending=False)
jan_df_delay_VendorID.iloc[:1]

,VendorID,Avg delay per VendorID
0,1,5.225513


Q4)What is the average trip duration across pickup locations?

In [47]:
Average_durationperLocation=jan_df.groupby('PULocationID')['expected_duration_min'].mean().compute().reset_index(name='Average Trip duration in minutes')

In [48]:
Average_durationperLocation=Average_durationperLocation.sort_values(by='Average Trip duration in minutes',ascending=False)

In [49]:
Average_durationperLocation

,PULocationID,Average Trip duration in minutes
248,257,152.334783
252,261,65.828737
13,14,61.740000
4,5,54.589286
110,118,48.417000
...,...,...
184,193,2.480258
28,30,1.470000
95,99,0.090000
107,115,0.000000


## Passenger Insights & Patterns
Purpose: Understand passenger habits and value.

In [50]:
avg_passenger_per_trip=jan_df['passenger_count'].mean().compute()

In [51]:
round(avg_passenger_per_trip,2)

np.float64(1.36)

Q1)Which locations have the highest number of unique trips (passenger demand clusters)?


To find locations with the highest number of unique trips (indicating passenger demand clusters), Im using PULocationID (pickup location) and count how many trips originated from each.

In [52]:
pickup_demand = jan_df.groupby('PULocationID').size().compute().reset_index(name='trip_count')
pickup_demand_sorted = pickup_demand.sort_values(by='trip_count', ascending=False)

print(pickup_demand_sorted.head(10))  # Top 10 high-demand pickup locations


     PULocationID  trip_count
124           132      160024
228           237      148074
227           236      138391
153           161      135415
177           186      109226
154           162      105333
134           142      100226
221           230       98988
130           138       89186
162           170       88346


Q2)What is the average total fare per passenger count group?


In [85]:
jan_df['passenger_count'].tail()

,passenger_count
3066761,1.0
3066762,1.0
3066763,1.0
3066764,1.0
3066765,1.0


In [72]:
jan_df['passenger_count'].isna().sum().compute()

np.int64(0)

In [70]:
mean_passenger=round(jan_df['passenger_count'].mean().compute(),0)

In [71]:
jan_df['passenger_count']=jan_df['passenger_count'].fillna(mean_passenger)

In [73]:
Avg_FarePERPassenger_count=jan_df.groupby('passenger_count')['total_amount'].mean().compute().reset_index(name='Avg _Fare/Passenger count')

In [75]:
Avg_FarePERPassenger_count.sort_values(by='Avg _Fare/Passenger count',ascending=False)

,passenger_count,Avg _Fare/Passenger count
8,8.0,99.336923
9,9.0,92.250000
7,7.0,85.111667
4,4.0,29.611602
2,2.0,29.313098
3,3.0,28.475520
5,5.0,26.587692
6,6.0,26.558484
1,1.0,26.526010
0,0.0,24.162124


 Q3) What times of day have the highest ride frequency?


In [76]:
jan_df['pickup_hr']=jan_df['tpep_pickup_datetime'].dt.hour

In [77]:
jan_rideFrequency = (
    jan_df['pickup_hr']
    .value_counts()
    .compute()
    .reset_index(name='Frequency')
    .rename(columns={'index': 'pickup_hour'})
    .sort_values(by='Frequency', ascending=False)
)


In [78]:
jan_rideFrequency

,pickup_hr,Frequency
18,18,215889
17,17,209492
15,15,196423
16,16,195977
19,19,192801
14,14,191602
13,13,178738
12,12,169858
20,20,165861
21,21,161547


Q4) What are the most common days/times for travel?


In [79]:
jan_df['pickup_day']=jan_df['tpep_pickup_datetime'].dt.day_name()

In [80]:
jan_commonday= (
    jan_df['pickup_day']
    .value_counts()
    .compute()
    .reset_index(name='common_day')
    .rename(columns={'index': 'pickup_day'})
    .sort_values(by='common_day', ascending=False)
)


In [81]:
jan_commonday

,pickup_day,common_day
5,Tuesday,490981
4,Thursday,441929
2,Saturday,441630
3,Sunday,436434
0,Friday,434381
6,Wednesday,416558
1,Monday,404805


 Most Common Day-Hour Combo:


In [82]:
jan_df['pickup_day_hour_AMPM'] = jan_df['tpep_pickup_datetime'].dt.strftime('%A %H:00')

day_hour_counts = (
    jan_df['pickup_day_hour_AMPM']
    .value_counts()
    .compute()
    .reset_index(name='Ride_Count')
    .rename(columns={'index': 'Day_Hour'})
    .sort_values(by='Ride_Count', ascending=False)
)

In [83]:
day_hour_counts

,pickup_day_hour_AMPM,Ride_Count
138,Tuesday 18:00,36405
137,Tuesday 17:00,34731
114,Thursday 18:00,34409
162,Wednesday 18:00,32941
135,Tuesday 15:00,32007
...,...,...
99,Thursday 03:00,1289
124,Tuesday 04:00,1225
148,Wednesday 04:00,1062
123,Tuesday 03:00,1030
